In [1]:
# Uncomment below to execute in workspace
%load_ext sql

In [2]:
import psycopg2
import etl
from resource_reader import *
from sql_queries import *

# 1. Load configuration paramater

In [3]:
config_file_name = 'dwh.yml'
rr = ResourceReader()
config = rr.config_reader(config_file_name)

CLUSTER_TYPE = config['CLUSTER']['TYPE']
NODE_NUMBER = int(config['CLUSTER']['NUMBER_OF_NODE'])
NODE_TYPE = config['CLUSTER']['NODE_TYPE']
CLUSTER_IDENTIFIER = config['CLUSTER']['CLUSTER_IDENTIFIER']

DB_NAME = config['CLUSTER']['DB_NAME']
DB_USERNAME = config['CLUSTER']['DB_USER']
DB_PASSWORD = config['CLUSTER']['DB_PASSWORD']
PORT = config['CLUSTER']['DB_PORT']

ACCESS_KEY_ID = config['AWS']['ACCESS_KEY_ID']
SECRET_ACCESS_KEY = config['AWS']['SECRET_ACCESS_KEY']
IAM_ROLE_NAME = config['IAM_ROLE']['NAME']

WAIT = int(config['CLUSTER']['DEFAULT_WAIT'])
TIMEOUT = int(config['CLUSTER']['TIMEOUT'])

Configs loaded!


# 2. Init AWS services (IAM, S3, EC2, and Redshift)

In [4]:
rr.setup_aws_services(ACCESS_KEY_ID, SECRET_ACCESS_KEY)

Establish IAM, S3, EC2, and Redshift services


## 2.1 Create IAM Role
### Specify <font color=red>IAM Role Name</font> in config parameter: `configs['IAM_ROLE']['NAME']`

In [5]:
# Uncomment to create the cluster
# Create IAM Role if it is unavailalbe
arn_test = rr.get_iam_role_arn(config)
if 'arn:aws:iam' not in arn_test:
    rr.create_iam_role(config)
# Apply default requirement policy `AmazonS3ReadOnlyAccess` arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess
rr.apply_policy(config)

role_name_test_for_AmazonS3ReadOnlyAccess3
IAM role ARN: [arn:aws:iam::587843818199:role/role_name_test_for_AmazonS3ReadOnlyAccess3]
Attach role policy successful


{'ResponseMetadata': {'RequestId': '7c64bed2-c9fb-42b1-9515-ba570e95aced',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '7c64bed2-c9fb-42b1-9515-ba570e95aced',
   'content-type': 'text/xml',
   'content-length': '212',
   'date': 'Wed, 24 Aug 2022 12:04:11 GMT'},
  'RetryAttempts': 0}}

## 2.2 Create Redshift Cluster
### a. Specify <font color=red>CLUSTER NAME</font> in config parameter: `configs['CLUSTER']['CLUSTER_IDENTIFIER']`

In [6]:
# Uncomment to create new Redshift Cluster
# response = rr.create_redshift_cluster(config)
#print(response)
# Bear with me while cluster is ready
# Cluster's state: [paused, resuming, Modifying... available]
rr.wait_until_cluster_status(CLUSTER_IDENTIFIER, 'available', WAIT, TIMEOUT)

[10] fetching retry...
Cluster state:  available


True

### b. Get information of created Redshift Cluster

In [7]:
# paused, resuming, available
status = 'available'
specs = rr.get_cluster_description(CLUSTER_IDENTIFIER)
CLUSTER_NAME = specs['ClusterIdentifier']
USERNAME = specs['MasterUsername']
DB_NAME = specs['DBName']
CLUSTER_ENDPOINT_URL = specs['Endpoint']['Address']
CLUSTER_ENDPOINT_PORT = specs['Endpoint']['Port']
ROLE_ARN = specs['IamRoles'][0]['IamRoleArn']
print('Cluster name: ', CLUSTER_NAME)
print('Cluster endpoint: ', CLUSTER_ENDPOINT_URL)
print('Cluster port: ', CLUSTER_ENDPOINT_PORT)
print('DB name: ', DB_NAME)
print('Username: ', USERNAME)

Cluster name:  redshift-cluster-useast
Cluster endpoint:  redshift-cluster-useast.cw5jqkebdvqv.us-east-1.redshift.amazonaws.com
Cluster port:  5439
DB name:  dev
Username:  student


### c. Configure incoming traffic to the cluster endpoint

In [8]:
rr.configure_incoming_tcp_traffic(config)

An error occurred (InvalidPermission.Duplicate) when calling the AuthorizeSecurityGroupIngress operation: the specified rule "peer: 0.0.0.0/0, TCP, from port: 5439, to port: 5439, ALLOW" already exists


## 2.3 Check Out Resources in AWS S3

### Load a few sample data sets to reduce consumption

In [9]:
rr.check_out_s3_resource(config)

'song_data/A/A/A/TRAAAAW128F429D538.json'
'song_data/A/A/A/TRAAABD128F429CF47.json'
'song_data/A/A/A/TRAAADZ128F9348C2E.json'
'song_data/A/A/A/TRAAAEF128F4273421.json'
'song_data/A/A/A/TRAAAFD128F92F423A.json'
SONG DATA COUNT:  71
'log_data/2018/11/2018-11-01-events.json'
'log_data/2018/11/2018-11-02-events.json'
'log_data/2018/11/2018-11-03-events.json'
'log_data/2018/11/2018-11-04-events.json'
'log_data/2018/11/2018-11-05-events.json'
LOG DATA COUNT:  30
'log_json_path.json'
{ 'jsonpaths': [ "$['artist']",
                 "$['auth']",
                 "$['firstName']",
                 "$['gender']",
                 "$['itemInSession']",
                 "$['lastName']",
                 "$['length']",
                 "$['level']",
                 "$['location']",
                 "$['method']",
                 "$['page']",
                 "$['registration']",
                 "$['sessionId']",
                 "$['song']",
                 "$['status']",
                 "$['t

## 3. Init Database Connection

In [10]:
conn_string = "host={} dbname={} user={} password={} port={}".format(
        CLUSTER_ENDPOINT_URL, DB_NAME, USERNAME,
        config['CLUSTER']['DB_PASSWORD'], CLUSTER_ENDPOINT_PORT)

conn = psycopg2.connect(conn_string)
cur = conn.cursor()
print('DB connected!')


DB connected!


## 4. Migrate staging data from S3 to staging tables on created Redshift Cluster

### 4.1 Migrate event logging, song data to staging tables

In [11]:
etl.migrate_staging_table_data(cur, conn)

Migrate staging data. Query [
    COPY staging_events
    FROM 's3://myeyesbucket.test/log_data'
    CREDENTIALS 'aws_iam_role=arn:aws:iam::587843818199:role/role_name_test_for_AmazonS3ReadOnlyAccess3'
    FORMAT AS JSON 's3://myeyesbucket.test/log_json_path.json'
    STATUPDATE ON
    REGION 'us-east-1';
]
Migrate staging data. Query [
    COPY staging_songs FROM 's3://myeyesbucket.test/song_data/'
    CREDENTIALS 'aws_iam_role=arn:aws:iam::587843818199:role/role_name_test_for_AmazonS3ReadOnlyAccess3'
    FORMAT AS JSON 'auto'
    ACCEPTINVCHARS AS '^'
    STATUPDATE ON
    REGION 'us-east-1';
]
Staging data migrated. DONE!


### 4.2 Populate analytical data into relevant tables

In [12]:
etl.insert_analytical_table_data(cur, conn)

Insert data. Query [
    INSERT INTO songplays (start_time, user_id, level, song_id, artist_id,  session_id, location, user_agent)                
    SELECT DISTINCT 
        TIMESTAMP 'epoch' + se.ts/1000 * INTERVAL '1 second', userId, 
        level, song_id, artist_id, sessionId, location, userAgent
    FROM staging_songs ss 
    JOIN staging_events se ON (ss.artist_name = se.artist AND ss.title = se.song)
    WHERE se.page = 'NextSong';
]
Insert data. Query [
    INSERT INTO users (user_id, first_name, last_name, gender, level)
    SELECT DISTINCT userId, firstName, lastName, gender, level 
    FROM staging_events
    WHERE page ='NextSong' AND userId IS NOT NULL;
]
Insert data. Query [
    INSERT INTO songs (song_id, song_title, artist_id, year, duration)
    SELECT DISTINCT song_id, title, artist_id, year, duration
    FROM staging_songs
    WHERE song_id IS NOT NULL;
]
Insert data. Query [
    INSERT INTO artists (artist_id, artist_name, artist_location, artist_latitude, artist

### 4.3 Inquiry migrated data

In [20]:
cur.execute('SELECT COUNT(*) FROM staging_events')
print(cur.fetchall()[0][0])

136952


In [21]:
query = 'SELECT COUNT(*) FROM staging_songs'
cur.execute(query)
print(cur.fetchall()[0][0])

782


In [27]:
query = 'SELECT COUNT(*) FROM users'
cur.execute(query)
print(cur.fetchall()[0][0])

2080


In [29]:
query = 'SELECT COUNT(*) FROM artists'
cur.execute(query)
print(cur.fetchall()[0][0])

451


In [30]:
query = 'SELECT COUNT(*) FROM time'
cur.execute(query)
print(cur.fetchall()[0][0])

5


In [31]:
query = 'SELECT COUNT(*) FROM songplays'
cur.execute(query)
print(cur.fetchall()[0][0])

21


## 5. Tear down

In [ ]:
# Uncomment to delete created Redshift Cluster
# rr.tear_down_redshift_cluster(config)